# Projeto INFNET -
## Projeto utilizando API do Foursquare
### Objetivo: 

## Importando as bibliotecas

In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from google.colab import drive

import json
import requests
import folium
from pandas.io.json import json_normalize


## Obtenção de dados
### Definição dos bairros a serem analisados
Os bairros foram definidos a partir de uma lista encontrada na internet com os 10 melhores bairros para se morar e outros bairros, que sei que também são requisitados, foram incluídos para a base. A grande maioria desse bairros fica na região Centro-sul de Belo Horizonte

In [4]:
dados = pd.DataFrame(
        [('Anchieta', -19.949453323219245, -43.92700801773141),
         ('Belvedere', -19.972559218057427, -43.93718789026611), 
         ('Centro', -19.919279099643592, -43.939315609652134),
         ('Cidade Jardim', -19.938641684001976, -43.950640230590984),
         ('Cidade Nova', -19.890068184185246, -43.924772018135),
         ('Castelo', -19.88213333487584, -43.999206085545126), 
         ('Gutierrez', -19.93678462997389, -43.95771548553014), 
         ('Buritis', -19.97563516763052, -43.96828714846663),
         ('Prado', -19.922907168023617, -43.960709744404966), 
         ('Lourdes', -19.93049620714053, -43.94296904336832), 
         ('Vila da Serra', -19.9546793, -43.9154242), 
         ('Floresta', -19.916406188338616, -43.92885909887027),
         ('Funcionários', -19.931827768132536, -43.92856347797582),
         ('Ouro Preto', -19.8736478717947, -43.98513618425674), 
         ('Santa Tereza', -19.91578247666307, -43.91675236069356), 
         ('Savassi', -19.93614741061873, -43.93349802745276),
         ('Santo Antônio', -19.94462581113818, -43.94392639104507),
         ('Santo Agostinho', -19.92917086904201, -43.95068392066596), 
         ('Santa Efigênia', -19.923409948392948, -43.92061174310462), 
         ('Sion', -19.953112178597326, -43.934422374007475)], columns=['Bairro', 'Lat', 'Lng'])

In [5]:
bh_map = folium.Map(location=[-19.9494271, -43.9269861],tiles='OpenStreetMap',  zoom_start=12)

In [8]:
for lat, lng, label in zip(dados['Lat'], dados['Lng'], dados['Bairro']):
  label = folium.Popup(label, parse_html=True)
  folium.CircleMarker(
      [lat, lng],
      radius=5,
      color='blue',
      fill=True,
      fill_opacity=0.5,
      parse_html=False).add_to(bh_map)

bh_map

Mapa criado com a biblioteca folium de Belo Horizonte, onde os pontos azuis representam locais próximos ao "centro" dos bairros escolhidos.<br><br><br>

### Conexão com API do FourSquare para obter locais próximos dos bairros
A distância é calculada a partir dos pontos azuis, com raio de 1000 metros

In [9]:
# CLIENT_ID = 'QW4X1SP4U1W1DFBGDL1DQHTRT3LIXF2OPA5K2M1S3D05SFR1'
# CLIENT_SECRET = 'ULOFDBVRBYQ1L0QSDTY1KCYZ033AG01SQS4N3WDDS34FJO1K'
# VERSION = 20220425
LIMIT = 50
radius = 1000

# cat_bar='13003%2C13029%2C13065%2C'
# cat_art='10000'
# cat_fuel = '19007'
# cat_essencial = '13002%2C15008%2C15014%2C12056%2C12057%2C12058%2C12059%2C12060%2C12061%2C12013%2C17035%2C17069%2C' #Drugstore, hospital, school, bakery, supermarket, etc


cat_casino = '10032%2C10052%2C13006%2C13009%2C13018%2C13029%2C19014' # bares, boates, hoteis

In [10]:
def getNearbyVenues(names, latitudes, longitudes):

  '''
    Função para conectar com a API do Foursquare, obter os dados 
    em formato json e tranformá-los em um dataframe
    '''

  df = pd.DataFrame()
  bairros = []

  for name, lat, lng in zip(names, latitudes, longitudes):

    # Criando o Request
    url = f'https://api.foursquare.com/v3/places/search?ll={lat},{lng}&radius={radius}&categories={cat_casino}&limit={LIMIT}'

    headers = {'Accept': 'application/json',
           'Authorization': 'fsq3AWxWY/26wzcU9YHkerFRNQXlWkwHBUsh8md+NYR8AJE='}

    # Making get request
    response = requests.get(url, headers=headers)
    json = response.json()
    
    df_mult = pd.json_normalize(json['results'])
    df_mult['Bairro'] = name
    df = pd.concat([df, df_mult])

  df.reset_index(inplace=True, drop=True)
  return(df)

Aplicando a função nos nossos dados dos bairros

In [11]:
# df = getNearbyVenues(dados['Bairro'], dados['Lat'], dados['Lng'])

In [93]:
# Criando uma cópia do df
# df_copy = df.copy()

In [12]:
df.head()

,fsq_id,categories,chains,distance,link,name,timezone,geocodes.main.latitude,geocodes.main.longitude,location.address,...,location.region,location.cross_street,location.neighborhood,location.address_extended,geocodes.roof.latitude,geocodes.roof.longitude,related_places.children,related_places.parent.fsq_id,related_places.parent.name,Bairro
0,50209ad729a6d2dc98e8f581,"[{'id': 10000, 'name': 'Arts and Entertainment...",[],195,/v3/places/50209ad729a6d2dc98e8f581,MIXING BAR - Premium Cocktails para Festas e E...,America/Sao_Paulo,-19.951138,-43.926094,"Rua Penafiel, 370",...,MG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Anchieta
1,542834a9498e550c059f6645,"[{'id': 13006, 'name': 'Beer Bar', 'icon': {'p...",[],264,/v3/places/542834a9498e550c059f6645,Bitaca Capetinga,America/Sao_Paulo,-19.948374,-43.929148,"Rua Francisco Deslandes, 104",...,MG,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Anchieta
2,536fa19a498ed80da22f94e0,"[{'id': 19014, 'name': 'Hotel', 'icon': {'pref...",[],794,/v3/places/536fa19a498ed80da22f94e0,e.Suítes Sion,America/Sao_Paulo,-19.955172,-43.931309,"Avenida Uruguai, 1010",...,MG,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Anchieta
3,53e2a66a498e415d67821083,"[{'id': 13018, 'name': 'Pub', 'icon': {'prefix...",[],539,/v3/places/53e2a66a498e415d67821083,Gilboa,America/Sao_Paulo,-19.946521,-43.931162,"Rua Pium-I, 772",...,MG,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Anchieta
4,526ae03e498e1e2092261522,"[{'id': 10000, 'name': 'Arts and Entertainment...",[],781,/v3/places/526ae03e498e1e2092261522,Jângal,America/Sao_Paulo,-19.942619,-43.929383,"Rua Outono, 523",...,MG,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Anchieta


## Pre processamento dos dados
###Extraindo as categorias do dicionário entes de salvar a base em um csv
As categorias são um dado importante, precisamos extrair da coluna 'categories'.<br>A coluna está em um formato de dicionário e cada registro pode ter até 3 categorias. Vou criar um lista em que cada registro possui uma lista com sua categorias.

In [13]:
items=list()
for venue in df['categories']:
  cat=list()
  for c in range(len(venue)):
    cat.append(venue[c]['name'])
  items.append(cat)

items

[['Arts and Entertainment', 'Cocktail Bar', 'Buffet'],
 ['Beer Bar', 'Gastropub'],
 ['Hotel'],
 ['Pub'],
 ['Arts and Entertainment', 'Pub', 'Restaurant'],
 ['Hostel', 'Hotel'],
 ['Bar', 'Brewery', 'Restaurant'],
 ['Beer Bar', 'Restaurant'],
 ['Beer Bar', 'Sports Bar', 'Restaurant'],
 ['Beer Bar'],
 ['Hotel'],
 ['Hotel'],
 ['Night Club', 'Bar', 'Restaurant'],
 ['Beer Bar'],
 ['Beer Bar', 'Restaurant'],
 ['Pub'],
 ['Pub'],
 ['Pub'],
 ['Cocktail Bar'],
 ['Beer Bar'],
 ['Brewery'],
 ['Beer Bar'],
 ['Beer Bar'],
 ['Brewery'],
 ['Hotel'],
 ['Beer Bar'],
 ['Hotel'],
 ['Brewery'],
 ['Brewery'],
 ['Night Club'],
 ['Hostel', 'Hotel', 'Inn'],
 ['Hotel'],
 ['Night Club', 'Bar'],
 ['Hotel'],
 ['Hotel'],
 ['Hotel'],
 ['Hostel', 'Hotel'],
 ['Night Club', 'Bar'],
 ['Hostel', 'Hotel'],
 ['Night Club', 'Bar'],
 ['Brewery'],
 ['Brewery', 'Restaurant'],
 ['Night Club'],
 ['Brewery', 'Restaurant'],
 ['Cocktail Bar'],
 ['Pub'],
 ['Brewery'],
 ['Night Club'],
 ['Hotel'],
 ['Beer Bar', 'Brazilian Restaurant']

### Criando colunas para as categorias
Serão criadas 3 novas colunas que serão preenchidas com as categorias de cada registro. Os locais que possuem 1 ou 2 categorias apenas, receberão um 'None' no lugar.

In [17]:
categ = pd.DataFrame(items, index= df.index, columns=['Cat1','Cat2','Cat3'])
categ.head()

,Cat1,Cat2,Cat3
0,Arts and Entertainment,Cocktail Bar,Buffet
1,Beer Bar,Gastropub,None
2,Hotel,None,None
3,Pub,None,None
4,Arts and Entertainment,Pub,Restaurant


In [19]:
df = pd.concat([df, categ], axis=1)
df.head(3)

,fsq_id,categories,chains,distance,link,name,timezone,geocodes.main.latitude,geocodes.main.longitude,location.address,...,location.address_extended,geocodes.roof.latitude,geocodes.roof.longitude,related_places.children,related_places.parent.fsq_id,related_places.parent.name,Bairro,Cat1,Cat2,Cat3
0,50209ad729a6d2dc98e8f581,"[{'id': 10000, 'name': 'Arts and Entertainment...",[],195,/v3/places/50209ad729a6d2dc98e8f581,MIXING BAR - Premium Cocktails para Festas e E...,America/Sao_Paulo,-19.951138,-43.926094,"Rua Penafiel, 370",...,NaN,NaN,NaN,NaN,NaN,NaN,Anchieta,Arts and Entertainment,Cocktail Bar,Buffet
1,542834a9498e550c059f6645,"[{'id': 13006, 'name': 'Beer Bar', 'icon': {'p...",[],264,/v3/places/542834a9498e550c059f6645,Bitaca Capetinga,America/Sao_Paulo,-19.948374,-43.929148,"Rua Francisco Deslandes, 104",...,NaN,NaN,NaN,NaN,NaN,NaN,Anchieta,Beer Bar,Gastropub,None
2,536fa19a498ed80da22f94e0,"[{'id': 19014, 'name': 'Hotel', 'icon': {'pref...",[],794,/v3/places/536fa19a498ed80da22f94e0,e.Suítes Sion,America/Sao_Paulo,-19.955172,-43.931309,"Avenida Uruguai, 1010",...,NaN,NaN,NaN,NaN,NaN,NaN,Anchieta,Hotel,None,None


### Excluindo lugares duplicados
'fsq_id' é o id único do local no Foursquare, vamos remover os duplicados.

In [20]:
df['fsq_id'].value_counts().head(5)

61019058b29040498c37b505    3
5c1e5bd261e53b002c58113f    3
4cdf49cec4f6a35d3a96cf6c    3
4d378997e6bdf04d9bfb5bb4    3
53879141498e7b711dca099f    3
Name: fsq_id, dtype: int64

In [22]:
# Verificando 
df[df['fsq_id'] == '61019058b29040498c37b505']

,fsq_id,categories,chains,distance,link,name,timezone,geocodes.main.latitude,geocodes.main.longitude,location.address,...,location.address_extended,geocodes.roof.latitude,geocodes.roof.longitude,related_places.children,related_places.parent.fsq_id,related_places.parent.name,Bairro,Cat1,Cat2,Cat3
342,61019058b29040498c37b505,"[{'id': 13029, 'name': 'Brewery', 'icon': {'pr...",[],933,/v3/places/61019058b29040498c37b505,Sátira,America/Sao_Paulo,-19.920458,-43.921032,"Avenida dos Andradas, 3000",...,NaN,NaN,NaN,NaN,4ccc7075ee23a14301171fa8,Boulevard Shopping,Floresta,Brewery,None,None
450,61019058b29040498c37b505,"[{'id': 13029, 'name': 'Brewery', 'icon': {'pr...",[],685,/v3/places/61019058b29040498c37b505,Sátira,America/Sao_Paulo,-19.920458,-43.921032,"Avenida dos Andradas, 3000",...,NaN,NaN,NaN,NaN,4ccc7075ee23a14301171fa8,Boulevard Shopping,Santa Tereza,Brewery,None,None
624,61019058b29040498c37b505,"[{'id': 13029, 'name': 'Brewery', 'icon': {'pr...",[],330,/v3/places/61019058b29040498c37b505,Sátira,America/Sao_Paulo,-19.920458,-43.921032,"Avenida dos Andradas, 3000",...,NaN,NaN,NaN,NaN,4ccc7075ee23a14301171fa8,Boulevard Shopping,Santa Efigênia,Brewery,None,None


In [23]:
# Dropando ids iguais
df.drop_duplicates(subset='fsq_id', keep='first', inplace=True)
df['fsq_id'].value_counts().head(5)

50209ad729a6d2dc98e8f581    1
f2721b46feda4864331965a8    1
4f22114ce4b006e5bfba998f    1
4bc778d593bdeee1e16b37ae    1
53c0ae07498ed50a2f0e02a8    1
Name: fsq_id, dtype: int64

In [26]:
df.tail()

,fsq_id,categories,chains,distance,link,name,timezone,geocodes.main.latitude,geocodes.main.longitude,location.address,...,location.address_extended,geocodes.roof.latitude,geocodes.roof.longitude,related_places.children,related_places.parent.fsq_id,related_places.parent.name,Bairro,Cat1,Cat2,Cat3
668,572cfa5a498eb7385c6e9006,"[{'id': 13029, 'name': 'Brewery', 'icon': {'pr...",[],620,/v3/places/572cfa5a498eb7385c6e9006,Growler Station Wals,America/Sao_Paulo,-19.954426,-43.940194,"Avenida Nossa Senhora, 1900",...,NaN,NaN,NaN,NaN,NaN,NaN,Sion,Brewery,None,None
670,4e5140466284416669a7923c,"[{'id': 10032, 'name': 'Night Club', 'icon': {...",[],757,/v3/places/4e5140466284416669a7923c,DJ Babi,America/Sao_Paulo,-19.957421,-43.940044,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Sion,Night Club,None,None
678,7f5e77f9bd3546b7aba082f1,"[{'id': 19014, 'name': 'Hotel', 'icon': {'pref...",[],481,/v3/places/7f5e77f9bd3546b7aba082f1,Hotel Rural Canto das Cachoeiras Ltda,NaN,-19.953583,-43.939009,"Avenida Nossa Senhora do Carmo, 1801",...,sl 102,NaN,NaN,NaN,NaN,NaN,Sion,Hotel,Inn,None
679,d5ad623140484d0dcfde731e,"[{'id': 19013, 'name': 'Hostel', 'icon': {'pre...",[],521,/v3/places/d5ad623140484d0dcfde731e,Apartamento do Escultor,NaN,-19.956793,-43.937526,"Rua Sta. Fé, 50",...,ap 402,NaN,NaN,NaN,NaN,NaN,Sion,Hostel,Hotel,None
684,2ade6eb00dee4a6743651edf,"[{'id': 19014, 'name': 'Hotel', 'icon': {'pref...",[],953,/v3/places/2ade6eb00dee4a6743651edf,Pousada Vila do Beco,NaN,-19.946842,-43.940655,"Rua s João Evangelista, 555",...,ap 301,NaN,NaN,NaN,NaN,NaN,Sion,Hotel,Inn,None


In [27]:
df.reset_index(inplace=True, drop=True)
df.head(3)

,fsq_id,categories,chains,distance,link,name,timezone,geocodes.main.latitude,geocodes.main.longitude,location.address,...,location.address_extended,geocodes.roof.latitude,geocodes.roof.longitude,related_places.children,related_places.parent.fsq_id,related_places.parent.name,Bairro,Cat1,Cat2,Cat3
0,50209ad729a6d2dc98e8f581,"[{'id': 10000, 'name': 'Arts and Entertainment...",[],195,/v3/places/50209ad729a6d2dc98e8f581,MIXING BAR - Premium Cocktails para Festas e E...,America/Sao_Paulo,-19.951138,-43.926094,"Rua Penafiel, 370",...,NaN,NaN,NaN,NaN,NaN,NaN,Anchieta,Arts and Entertainment,Cocktail Bar,Buffet
1,542834a9498e550c059f6645,"[{'id': 13006, 'name': 'Beer Bar', 'icon': {'p...",[],264,/v3/places/542834a9498e550c059f6645,Bitaca Capetinga,America/Sao_Paulo,-19.948374,-43.929148,"Rua Francisco Deslandes, 104",...,NaN,NaN,NaN,NaN,NaN,NaN,Anchieta,Beer Bar,Gastropub,None
2,536fa19a498ed80da22f94e0,"[{'id': 19014, 'name': 'Hotel', 'icon': {'pref...",[],794,/v3/places/536fa19a498ed80da22f94e0,e.Suítes Sion,America/Sao_Paulo,-19.955172,-43.931309,"Avenida Uruguai, 1010",...,NaN,NaN,NaN,NaN,NaN,NaN,Anchieta,Hotel,None,None


Salvando o df num arquivo csv para nao ficar precisando fazer requests

In [129]:
df.to_csv('venues_bh_cat_casino.csv')